<a href="https://colab.research.google.com/github/Soumya007-developer/DNA-Sequence-Analysis/blob/master/DNA_Sequencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Python packages like Biopython and Squiggle will be used when dealing with biological sequence data in Python.</h3>

Biopython installation

<h4>pip install biopython</h4>

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3MB 4.9MB/s 


Squiggle installation

<h4>pip install Squiggle</h4>

In [ ]:
!pip install Squiggle

  Created wheel for pyfaidx: filename=pyfaidx-0.5.9.1-cp36-none-any.whl size=25119 sha256=4c07e721ef29ad7d16cc921d4bbb671573af8400ecfcc9a83ce0e0a66b21a02b
  Stored in directory: /root/.cache/pip/wheels/57/cd/73/34f6bb5e5a8a4a014ee5bd14a951e12617bd7a4e10cb0df252
Successfully built pyfaidx


DNA sequence data usually are contained in a file format called “fasta” format. Fasta format is simply a single line prefixed by the greater than symbol that contains annotations and another line that contains the sequence:

<h5>“AAGGTGAGTGAAATCTCAACACGAGTATGGTTCTGAGAGTAGCTCTGTAACTCTGAGG”</h5>
The file can contain one or many DNA sequences. There are lots of other formats, but fasta is the most common.

We will use <strong>Bio.SeqIO</strong> from Biopython for parsing DNA sequence data(fasta) as it provides a simple uniform interface to input and output assorted sequence file formats.

In [ ]:
from Bio import SeqIO
for sequence in SeqIO.parse('/sample.fa', "fasta"):
    print(sequence.id)
    print(sequence.seq)
    print(len(sequence))

ENST00000435737.5
ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAATTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCCTTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTCCTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAAGACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCTGTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGAAGTCCTGGGGCTGACTCTCATTGTCTGGATTGACTGA
390
ENST00000419127.5
ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAATTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCCTTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTCCTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAAGACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCTGTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGACAAAGGCTGCTCTCAGTACTTCTATGCAGAGCATCTGTCTCTCCACTACCCGCTGGAGATTTCTGCAGCCTCAGGGAGGCTGATGTGTCACTTCAAGCTGGTGGCCATAGTGGGCTACCTGATTCGTCTCTCAATCAAGTCCATCCAAATCGAAGCCGACAACTGTGTCACTGACTCCCTGACCATTTACGACTCCCTTTTGCCCATCCGGAGCAG

There are **3 general approaches** to encode sequence data:

1.Ordinal encoding DNA Sequence

2.One-hot encoding DNA Sequence

3.DNA sequence as a “language”, known as k-mer counting 

**Ordinal encoding DNA sequence data**

We need to encode each nitrogen bases as an ordinal value. For example “ATGC” becomes [0.25, 0.5, 0.75, 1.0]. Any other base such as “N” can be a 0.

In [ ]:
#Ordinal encoding DNA sequence data¶
# function to convert a DNA sequence string to a numpy array
# converts to lower case, changes any non 'acgt' characters to 'n'
import numpy as np
import re
def string_to_array(seq_string):
    seq_string = seq_string.lower()
    seq_string = re.sub('[^acgt]', 'z', seq_string)
    seq_string = np.array(list(seq_string))
    return seq_string
    
# create a label encoder with 'acgtn' alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))


LabelEncoder()

Here is a function to encode a DNA sequence string as an ordinal vector. It returns a NumPy array with A=0.25, C=0.50, G=0.75, T=1.00, n=0.00.

In [ ]:
# function to encode a DNA sequence string as an ordinal vector
# returns a numpy vector with a=0.25, c=0.50, g=0.75, t=1.00, n=0.00
def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, lets say z
    return float_encoded

seq_test = 'TTCAGCCAGTG'
ordinal_encoder(string_to_array(seq_test))


array([1.  , 1.  , 0.5 , 0.25, 0.75, 0.5 , 0.5 , 0.25, 0.75, 1.  , 0.75])

**One-hot encoding DNA Sequence**

It is widely used in deep learning methods and lends itself well to algorithms like CNN.For instance, "ATGC" will become [0,0,0,1], [0,0,1,0], [0,1,0,0], [1,0,0,0]. Then, it is turned into a 2-D Array.

In [8]:
#One-hot encoding DNA sequence data¶
# function to one-hot encode a DNA sequence string
# non 'acgt' bases (n) are 0000
from sklearn.preprocessing import OneHotEncoder
def one_hot_encoder(seq_string):
    int_encoded = label_encoder.transform(seq_string)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)
    int_encoded = int_encoded.reshape(len(int_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(int_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded
#let’s try it out with a simple short sequence:

seq_test = 'GAATTCTCGAA'
one_hot_encoder(string_to_array(seq_test))

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

**K-mer counting**

A DNA sequence termed as "language" is know as K-mer counting.